Web Scraping Using Selenium

In [12]:
#andhra pradesh
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

# Initialize the Chrome driver
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/")
time.sleep(3)
driver.maximize_window()
driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")
time.sleep(2)

# store the data
bus_routes_list = []
link_list = []

def extract_data():
    # Extract data from the page
    bus_routes = driver.find_elements(By.CLASS_NAME, "route")
    for route in bus_routes:
        bus_routes_list.append(route.text)
    
    for link in bus_routes:
        href = link.get_attribute('href')
        link_list.append(href)

# Extract initial data
extract_data()

while True:
    try:
        # Identify and click the 'Next' button
        current_page = driver.find_element(By.CSS_SELECTOR, "div[class='DC_117_pageTabs DC_117_pageActive']")
        driver.execute_script("arguments[0].scrollIntoView();", current_page)

        next_page_number = int(current_page.text) + 1

        next_page = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="root"]/div/div[4]/div[12]/div[{next_page_number}]'))
        )
        next_page.click()

        time.sleep(10)  
        extract_data()  # Extract data from the next page
    except Exception as e:
        break  

all_data = []

for route, link in zip(bus_routes_list, link_list):
    driver.get(link)
    time.sleep(5)
    
    try:
        view_buses_buttons = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']"))
        )

        # Click each "View Buses" button using JavaScript
        for view_buses_button in view_buses_buttons:
            driver.execute_script("arguments[0].scrollIntoView(true);", view_buses_button)
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2) 
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        bus_names = driver.find_elements(By.CSS_SELECTOR, 'div[class="travels lh-24 f-bold d-color"]')
        bus_types = driver.find_elements(By.CSS_SELECTOR, 'div[class="bus-type f-12 m-top-16 l-color evBus"]')
        starting_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="dp-time f-19 d-color f-bold"]')
        drop_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="bp-time f-19 d-color disp-Inline"]')
        ratings = driver.find_elements(By.CSS_SELECTOR, 'div[class="rating-sec lh-24"]')
        fares = driver.find_elements(By.CSS_SELECTOR, 'div[class="fare d-block"]')
        seats_available = driver.find_elements(By.CSS_SELECTOR, "div[class='seat-left m-top-30']")
        bus_durations = driver.find_elements(By.CSS_SELECTOR, 'div[class="dur l-color lh-24"]')

        list_lengths = [len(bus_names), len(bus_types), len(starting_times), len(drop_times), len(ratings), len(fares), len(seats_available), len(bus_durations)]
        min_length = min(list_lengths)

        if min_length == 0:
            # Skipping route  missing data

            continue

        for i in range(min_length):
            data= {
                "Busroutes": route,
                "Bus Routes Link": link,
                "Bus Name": bus_names[i].text if i < len(bus_names) else "NA",
                "Bus_Type": bus_types[i].text if i < len(bus_types) else "NA",
                "Starting_Time": starting_times[i].text if i < len(starting_times) else "NA",
                "Rating": ratings[i].text if i < len(ratings) and ratings[i].text else "NA",
                "Dropping_Time": drop_times[i].text if i < len(drop_times) else "NA",
                "Fare": fares[i].text if i < len(fares) else "NA",
                "Available Seats": seats_available[i].text if i < len(seats_available) and seats_available[i].text else "NA",
                "Duration": bus_durations[i].text if i < len(bus_durations) else "NA"
            }
            all_data.append(data)

    except Exception as e:
        pass

driver.quit()
print("successfully completed")





successfully completed


In [7]:
data1= pd.DataFrame(all_data)
data1



,Busroutes,Bus Routes Link,Bus Name,Bus_Type,Starting_Time,Rating,Dropping_Time,Fare,Available Seats,Duration
0,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 9356,Super Luxury (Non AC Seater 2+2 Push Back),11:30,4.0,18:05,INR 446,46 Seats available,06h 35m
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3858,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),11:30,3.6,16:30,INR 720,33 Seats available,05h 00m
2,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3601,Super Luxury (Non AC Seater 2+2 Push Back),12:00,2.5,20:20,INR 446,28 Seats available,08h 20m
3,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 35269,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),12:15,3.5,18:25,INR 683,43 Seats available,06h 10m
4,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3515,Super Luxury (Non AC Seater 2+2 Push Back),12:30,2.0,20:05,INR 446,25 Seats available,07h 35m
...,...,...,...,...,...,...,...,...,...,...
3894,Anantapur (andhra pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/ananthapur-t...,Om Sri Vinayaka Travels,A/C Seater / Sleeper (2+1),23:59,4.4,07:25,INR 800,36 Seats available,07h 26m
3895,Anantapur (andhra pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/ananthapur-t...,S.L Travels,A/C Sleeper (2+1),23:45,4.0,06:35,INR 1100,46 Seats available,06h 50m
3896,Anantapur (andhra pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/ananthapur-t...,S.L Travels,A/C Sleeper (2+1),23:45,4.0,06:35,INR 1000,48 Seats available,06h 50m
3897,Anantapur (andhra pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/ananthapur-t...,Orange Tours And Travels,A/C Seater / Sleeper (2+1),22:45,3.9,05:30,INR 850,24 Seats available,06h 45m


In [13]:
#kerala
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

# Initialize the Chrome driver
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/")
time.sleep(3)
driver.maximize_window()
driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")
time.sleep(2)

# Lists to store data
bus_routes_list = []
link_list = []

def extract_data():
    # Extract data from the page
    bus_routes = driver.find_elements(By.CLASS_NAME, "route")
    for route in bus_routes:
        bus_routes_list.append(route.text)
    
    for link in bus_routes:
        href = link.get_attribute('href')
        link_list.append(href)

# Extract initial data
extract_data()

while True:
    try:
        # Identify and click the 'Next' button
        current_page = driver.find_element(By.CSS_SELECTOR, "div[class='DC_117_pageTabs DC_117_pageActive']")
        driver.execute_script("arguments[0].scrollIntoView();", current_page)

        next_page_number = int(current_page.text) + 1

        next_page = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="root"]/div/div[4]/div[12]/div[{next_page_number}]'))
        )
        next_page.click()

        time.sleep(10) 
        extract_data()  # Extract data from the new page
    except Exception as e:
        break  # Exit the loop if the next button is not found or an error occurs

all_data = []

for route, link in zip(bus_routes_list, link_list):
    driver.get(link)
    time.sleep(5)
    
    try:
        view_buses_buttons = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']"))
        )

        # Click each "View Buses" button using JavaScript
        for view_buses_button in view_buses_buttons:
            driver.execute_script("arguments[0].scrollIntoView(true);", view_buses_button)
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2) 
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        bus_names = driver.find_elements(By.CSS_SELECTOR, 'div[class="travels lh-24 f-bold d-color"]')
        bus_types = driver.find_elements(By.CSS_SELECTOR, 'div[class="bus-type f-12 m-top-16 l-color evBus"]')
        starting_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="dp-time f-19 d-color f-bold"]')
        drop_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="bp-time f-19 d-color disp-Inline"]')
        ratings = driver.find_elements(By.CSS_SELECTOR, 'div[class="rating-sec lh-24"]')
        fares = driver.find_elements(By.CSS_SELECTOR, 'div[class="fare d-block"]')
        seats_available = driver.find_elements(By.CSS_SELECTOR, "div[class='seat-left m-top-30']")
        bus_durations = driver.find_elements(By.CSS_SELECTOR, 'div[class="dur l-color lh-24"]')

        list_lengths = [len(bus_names), len(bus_types), len(starting_times), len(drop_times), len(ratings), len(fares), len(seats_available), len(bus_durations)]
        min_length = min(list_lengths)

        if min_length == 0:
            continue

        for i in range(min_length):
            data = {
                "Busroutes": route,
                "Bus Routes Link": link,
                "Bus Name": bus_names[i].text if i < len(bus_names) else "NA",
                "Bus_Type": bus_types[i].text if i < len(bus_types) else "NA",
                "Starting_Time": starting_times[i].text if i < len(starting_times) else "NA",
                "Rating": ratings[i].text if i < len(ratings) and ratings[i].text else "NA",
                "Dropping_Time": drop_times[i].text if i < len(drop_times) else "NA",
                "Fare": fares[i].text if i < len(fares) else "NA",
                "Available Seats": seats_available[i].text if i < len(seats_available) and seats_available[i].text else "NA",
                "Duration": bus_durations[i].text if i < len(bus_durations) else "NA"
            }
            all_data.append(data)

    except Exception as e:
        pass

driver.quit()
print("Successfully completed")


Successfully completed


In [8]:

data_2= pd.DataFrame(all_data)
data_2


,Busroutes,Bus Routes Link,Bus Name,Bus_Type,Starting_Time,Rating,Dropping_Time,Fare,Available Seats,Duration
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,4.3,22:50,INR 627,15 Seats available,08h 35m
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,4.3,00:01,INR 620,30 Seats available,09h 01m
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 97,Swift Deluxe Non AC Air Bus (2+2),16:02,2.8,00:50,INR 567,38 Seats available,08h 48m
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 171,Swift Deluxe Non AC Air Bus (2+2),16:16,4.7,01:20,INR 640,31 Seats available,09h 04m
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 553,Super Express Non AC Seater Air Bus (2+2),16:45,2.0,01:25,INR 640,33 Seats available,08h 40m
...,...,...,...,...,...,...,...,...,...,...
468,Kozhikode to Kalpetta (kerala),https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 897,Super Fast Non AC Seater (2+3),15:01,4.1,17:11,INR 108,16 Seats available,02h 10m
469,Kozhikode to Kalpetta (kerala),https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 708,Super Fast Non AC Seater (2+3),16:01,4.1,17:50,INR 111,49 Seats available,01h 49m
470,Kalpetta (kerala) to Kozhikode,https://www.redbus.in/bus-tickets/kalpetta-to-...,KSRTC (Kerala) - 323,Super Fast Non AC Seater (2+3),15:30,2.4,17:25,INR 111,28 Seats available,01h 55m
471,Kalpetta (kerala) to Kozhikode,https://www.redbus.in/bus-tickets/kalpetta-to-...,KSRTC (Kerala) - 48,Swift Deluxe Non AC Air Bus (2+2),17:10,3.8,19:00,INR 145,20 Seats available,01h 50m


In [14]:
#telangana
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

# Initialize the Chrome driver
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/")
time.sleep(3)
driver.maximize_window()
driver.get("https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile")
time.sleep(2)

# Lists to store data
bus_routes_list = []
link_list = []

def extract_data():
    # Extract data from the page
    bus_routes = driver.find_elements(By.CLASS_NAME, "route")
    for route in bus_routes:
        bus_routes_list.append(route.text)
    
    for link in bus_routes:
        href = link.get_attribute('href')
        link_list.append(href)

# Extract initial data
extract_data()

while True:
    try:
        # Identify and click the 'Next' button
        current_page = driver.find_element(By.CSS_SELECTOR, "div[class='DC_117_pageTabs DC_117_pageActive']")
        driver.execute_script("arguments[0].scrollIntoView();", current_page)

        next_page_number = int(current_page.text) + 1

        next_page = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="root"]/div/div[4]/div[12]/div[{next_page_number}]'))
        )
        next_page.click()

        time.sleep(10)  # Wait for the next page to load
        extract_data()  
    except Exception as e:

        break  # Exit the loop if the next button is not found or an error occurs

all_data = []

for route, link in zip(bus_routes_list, link_list):
    driver.get(link)
    time.sleep(5)
    
    try:
        view_buses_buttons = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']"))
        )

        # Click each "View Buses" button using JavaScript
        for view_buses_button in view_buses_buttons:
            driver.execute_script("arguments[0].scrollIntoView(true);", view_buses_button)
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2) 
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        bus_names = driver.find_elements(By.CSS_SELECTOR, 'div[class="travels lh-24 f-bold d-color"]')
        bus_types = driver.find_elements(By.CSS_SELECTOR, 'div[class="bus-type f-12 m-top-16 l-color evBus"]')
        starting_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="dp-time f-19 d-color f-bold"]')
        drop_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="bp-time f-19 d-color disp-Inline"]')
        ratings = driver.find_elements(By.CSS_SELECTOR, 'div[class="rating-sec lh-24"]')
        fares = driver.find_elements(By.CSS_SELECTOR, 'div[class="fare d-block"]')
        seats_available = driver.find_elements(By.CSS_SELECTOR, "div[class='seat-left m-top-30']")
        bus_durations = driver.find_elements(By.CSS_SELECTOR, 'div[class="dur l-color lh-24"]')

        list_lengths = [len(bus_names), len(bus_types), len(starting_times), len(drop_times), len(ratings), len(fares), len(seats_available), len(bus_durations)]
        min_length = min(list_lengths)

        if min_length == 0:
            continue

        for i in range(min_length):
            data= {
                "Busroutes": route,
                "Bus Routes Link": link,
                "Bus Name": bus_names[i].text if i < len(bus_names) else "NA",
                "Bus_Type": bus_types[i].text if i < len(bus_types) else "NA",
                "Starting_Time": starting_times[i].text if i < len(starting_times) else "NA",
                "Rating": ratings[i].text if i < len(ratings) and ratings[i].text else "NA",
                "Dropping_Time": drop_times[i].text if i < len(drop_times) else "NA",
                "Fare": fares[i].text if i < len(fares) else "NA",
                "Available Seats": seats_available[i].text if i < len(seats_available) and seats_available[i].text else "NA",
                "Duration": bus_durations[i].text if i < len(bus_durations) else "NA"
            }
            all_data.append(data)

    except Exception as e:
        pass

driver.quit()
print("Successfully completed")


Successfully completed


In [9]:


data_3=pd.DataFrame(all_data)
data_3


,Busroutes,Bus Routes Link,Bus Name,Bus_Type,Starting_Time,Rating,Dropping_Time,Fare,Available Seats,Duration
0,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8609,RAJDHANI (A.C. Semi Sleeper),12:10,3.3,16:10,INR 389,35 Seats available,04h 00m
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8918,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",12:30,3.5,16:50,INR 320,34 Seats available,04h 20m
2,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8681,RAJADHANI AC (CONVERTED METRO LUXURY),13:00,2.9,17:00,INR 389,28 Seats available,04h 00m
3,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8617,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",13:20,4.9,17:30,INR 320,29 Seats available,04h 10m
4,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8736,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",13:30,2.3,17:50,INR 320,29 Seats available,04h 20m
...,...,...,...,...,...,...,...,...,...,...
1621,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,True Bus,Volvo Multi Axle B9R A/C Sleeper (2+1),22:30,3.8,09:00,INR 4000,36 Seats available,10h 30m
1622,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,Tranzindia Travels,Bharat Benz A/C Sleeper (2+1),21:45,3.6,08:50,INR 900,15 Seats available,11h 05m
1623,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,Zingbus Plus,A/C Sleeper (2+1),22:35,3.5,07:50,764,26 Seats available,09h 15m
1624,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,VEGA BUS,A/C Seater / Sleeper (2+1),22:20,3.5,09:45,3325,1 Seat available,11h 25m


In [16]:
#rajasthan
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

# Initialize the Chrome driver
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/")
time.sleep(3)
driver.maximize_window()
driver.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile")
time.sleep(2)

# Lists to store data
bus_routes_list = []
link_list = []

def extract_data():
    # Extract data from the page
    bus_routes = driver.find_elements(By.CLASS_NAME, "route")
    for route in bus_routes:
        bus_routes_list.append(route.text)
    
    for link in bus_routes:
        href = link.get_attribute('href')
        link_list.append(href)

# Extract initial data
extract_data()

while True:
    try:
        # Identify and click the 'Next' button
        current_page = driver.find_element(By.CSS_SELECTOR, "div[class='DC_117_pageTabs DC_117_pageActive']")
        driver.execute_script("arguments[0].scrollIntoView();", current_page)

        next_page_number = int(current_page.text) + 1

        next_page = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="root"]/div/div[4]/div[12]/div[{next_page_number}]'))
        )
        next_page.click()

        time.sleep(10)  
        extract_data()  # Extract data from the next page
    except Exception as e:
        break  # Exit the loop if the next button is not found or an error occurs

all_data = []

for route, link in zip(bus_routes_list, link_list):
    driver.get(link)
    time.sleep(5)
    
    try:
        view_buses_buttons = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']"))
        )

        # Click each "View Buses" button using JavaScript
        for view_buses_button in view_buses_buttons:
            driver.execute_script("arguments[0].scrollIntoView(true);", view_buses_button)
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2) 
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        bus_names = driver.find_elements(By.CSS_SELECTOR, 'div[class="travels lh-24 f-bold d-color"]')
        bus_types = driver.find_elements(By.CSS_SELECTOR, 'div[class="bus-type f-12 m-top-16 l-color evBus"]')
        starting_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="dp-time f-19 d-color f-bold"]')
        drop_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="bp-time f-19 d-color disp-Inline"]')
        ratings = driver.find_elements(By.CSS_SELECTOR, 'div[class="rating-sec lh-24"]')
        fares = driver.find_elements(By.CSS_SELECTOR, 'div[class="fare d-block"]')
        seats_available = driver.find_elements(By.CSS_SELECTOR, "div[class='seat-left m-top-30']")
        bus_durations = driver.find_elements(By.CSS_SELECTOR, 'div[class="dur l-color lh-24"]')

        list_lengths = [len(bus_names), len(bus_types), len(starting_times), len(drop_times), len(ratings), len(fares), len(seats_available), len(bus_durations)]
        min_length = min(list_lengths)

        if min_length == 0:
            continue

        for i in range(min_length):
            data_4= {
                "Busroutes": route,
                "Bus Routes Link": link,
                "Bus Name": bus_names[i].text if i < len(bus_names) else "NA",
                "Bus_Type": bus_types[i].text if i < len(bus_types) else "NA",
                "Starting_Time": starting_times[i].text if i < len(starting_times) else "NA",
                "Rating": ratings[i].text if i < len(ratings) and ratings[i].text else "NA",
                "Dropping_Time": drop_times[i].text if i < len(drop_times) else "NA",
                "Fare": fares[i].text if i < len(fares) else "NA",
                "Available Seats": seats_available[i].text if i < len(seats_available) and seats_available[i].text else "NA",
                "Duration": bus_durations[i].text if i < len(bus_durations) else "NA"
            }
            all_data.append(data)

    except Exception as e:
        pass


# Close the driver
driver.quit()

print("Successfully completed")


Successfully completed


In [10]:

data_4=  pd.DataFrame(all_data)
data_4

,Busroutes,Bus Routes Link,Bus Name,Bus_Type,Starting_Time,Rating,Dropping_Time,Fare,Available Seats,Duration
0,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 178737,Express Non AC Seater 2+3,15:30,4.2,20:05,INR 229,37 Seats available,04h 35m
1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 190572,Express Non AC Seater 2+3,16:15,4.2,21:00,INR 229,42 Seats available,04h 45m
2,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 175650,Super Luxury Volvo AC Seater Pushback 2+2,16:30,4.0,20:30,INR 460,37 Seats available,04h 00m
3,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 160785,Super Luxury Volvo AC Seater Pushback 2+2,17:30,4.1,21:30,INR 460,23 Seats available,04h 00m
4,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 190951,Express Non AC Seater 2+3,18:00,4.2,22:30,INR 229,40 Seats available,04h 30m
...,...,...,...,...,...,...,...,...,...,...
523,Bikaner to Sikar,https://www.redbus.in/bus-tickets/bikaner-to-s...,Salasar Travels,A/C Sleeper (2+1),22:00,3.0,02:00,484,34 Seats available,04h 00m
524,Bikaner to Sikar,https://www.redbus.in/bus-tickets/bikaner-to-s...,Milan Travels,A/C Seater / Sleeper (2+1),22:30,4.1,02:30,INR 476,42 Seats available,04h 00m
525,Bikaner to Sikar,https://www.redbus.in/bus-tickets/bikaner-to-s...,RATHORE TRAVELS AGENCY,A/C Seater / Sleeper (2+1),22:30,3.5,02:59,INR 350,34 Seats available,04h 29m
526,Bikaner to Sikar,https://www.redbus.in/bus-tickets/bikaner-to-s...,Maharaja Travels,A/C Seater / Sleeper (2+1),22:30,3.2,02:45,INR 333,34 Seats available,04h 15m


In [31]:
#south bengal
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

# Initialize the Chrome driver
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/")
time.sleep(3)
driver.maximize_window()
driver.get("https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile")
time.sleep(2)

# Lists to store data
bus_routes_list = []
link_list = []

def extract_data():
    # Extract data from the page
    bus_routes = driver.find_elements(By.CLASS_NAME, "route")
    for route in bus_routes:
        bus_routes_list.append(route.text)
    
    for link in bus_routes:
        href = link.get_attribute('href')
        link_list.append(href)

# Extract initial data
extract_data()

while True:
    try:
        # Identify and click the 'Next' button
        current_page = driver.find_element(By.CSS_SELECTOR, "div[class='DC_117_pageTabs DC_117_pageActive']")
        driver.execute_script("arguments[0].scrollIntoView();", current_page)

        next_page_number = int(current_page.text) + 1

        next_page = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="root"]/div/div[4]/div[12]/div[{next_page_number}]'))
        )
        next_page.click()

        time.sleep(10)  
        extract_data()  # Extract data from the next page
    except Exception as e:
        break  # Exit the loop if the next button is not found or an error occurs

all_data = []

for route, link in zip(bus_routes_list, link_list):
    driver.get(link)
    time.sleep(5)
    
    try:
        view_buses_buttons = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']"))
        )

        # Click each "View Buses" button using JavaScript
        for view_buses_button in view_buses_buttons:
            driver.execute_script("arguments[0].scrollIntoView(true);", view_buses_button)
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2) 
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        bus_names = driver.find_elements(By.CSS_SELECTOR, 'div[class="travels lh-24 f-bold d-color"]')
        bus_types = driver.find_elements(By.CSS_SELECTOR, 'div[class="bus-type f-12 m-top-16 l-color evBus"]')
        starting_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="dp-time f-19 d-color f-bold"]')
        drop_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="bp-time f-19 d-color disp-Inline"]')
        ratings = driver.find_elements(By.CSS_SELECTOR, 'div[class="rating-sec lh-24"]')
        fares = driver.find_elements(By.CSS_SELECTOR, 'div[class="fare d-block"]')
        seats_available = driver.find_elements(By.CSS_SELECTOR, "div[class='seat-left m-top-30']")
        bus_durations = driver.find_elements(By.CSS_SELECTOR, 'div[class="dur l-color lh-24"]')

        list_lengths = [len(bus_names), len(bus_types), len(starting_times), len(drop_times), len(ratings), len(fares), len(seats_available), len(bus_durations)]
        min_length = min(list_lengths)

        if min_length == 0:
            continue

        for i in range(min_length):
            data= {
                "Busroutes": route,
                "Bus Routes Link": link,
                "Bus Name": bus_names[i].text if i < len(bus_names) else "NA",
                "Bus_Type": bus_types[i].text if i < len(bus_types) else "NA",
                "Starting_Time": starting_times[i].text if i < len(starting_times) else "NA",
                "Rating": ratings[i].text if i < len(ratings) and ratings[i].text else "NA",
                "Dropping_Time": drop_times[i].text if i < len(drop_times) else "NA",
                "Fare": fares[i].text if i < len(fares) else "NA",
                "Available Seats": seats_available[i].text if i < len(seats_available) and seats_available[i].text else "NA",
                "Duration": bus_durations[i].text if i < len(bus_durations) else "NA"
            }
            all_data.append(data)

    except Exception as e:
        pass


# Close the driver
driver.quit()

print("Successfully completed")

Successfully completed


In [11]:

data_5= pd.Dataframe(all_data)
data_5


,Busroutes,Bus Routes Link,Bus Name,Bus_Type,Starting_Time,Rating,Dropping_Time,Fare,Available Seats,Duration
0,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...,SBSTC-BURDWAN - KARUNAMOYEE - 15:10 (BWN DEPOT...,Non AC Seater (2+3),15:10,4.4,17:40,INR 100,36 Seats available,02h 30m
1,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...,SBSTC-KOLKATA - HALDIA - NON STOP - 15:30 (HAL...,Non AC Seater (2+3),15:30,4.6,18:30,INR 135,36 Seats available,03h 00m
2,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...,SBSTC-KARUNAMOYEE - BANKURA - A/C - 15:15 (BNK...,AC Seater (2+2),15:15,2.8,19:00,INR 300,24 Seats available,03h 45m
3,Digha to Kolkata,https://www.redbus.in/bus-tickets/digha-to-kol...,WBTC (CTC) Joka - Tollygunge - Digha - 99|15:30,Non AC Seater (2+3),15:30,3.6,20:45,INR 151,25 Seats available,05h 15m
4,Digha to Kolkata,https://www.redbus.in/bus-tickets/digha-to-kol...,WBTC (CTC) MADHYAMGRAM - DIGHA via Belghoria -...,Non AC Seater (2+3),16:00,2.3,20:30,INR 147,27 Seats available,04h 30m
...,...,...,...,...,...,...,...,...,...,...
168,Siliguri to Malda,https://www.redbus.in/bus-tickets/siliguri-to-...,Shyamoli Paribahan Pvt Ltd,Scania Multi-Axle AC Semi Sleeper (2+2),18:20,3.9,23:20,INR 500,14 Seats available,05h 00m
169,Siliguri to Malda,https://www.redbus.in/bus-tickets/siliguri-to-...,Express Line,Volvo 9600 Multi-Axle A/C Sleeper (2+1),18:30,4.6,23:45,999,50 Seats available,05h 15m
170,Siliguri to Malda,https://www.redbus.in/bus-tickets/siliguri-to-...,Greenline,Volvo 9600 Multi-Axle A/C Sleeper (2+1),19:00,5.0,23:59,INR 1100,33 Seats available,04h 59m
171,Siliguri to Malda,https://www.redbus.in/bus-tickets/siliguri-to-...,Modern Travels,Bharat Benz A/C Seater /Sleeper (2+1),19:00,3.6,23:55,INR 825,33 Seats available,04h 55m


In [33]:
#himachal

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

# Initialize the Chrome driver
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/")
time.sleep(3)
driver.maximize_window()
driver.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile")
time.sleep(2)

# Lists to store data
bus_routes_list = []
link_list = []

def extract_data():
    # Extract data from the page
    bus_routes = driver.find_elements(By.CLASS_NAME, "route")
    for route in bus_routes:
        bus_routes_list.append(route.text)
    
    for link in bus_routes:
        href = link.get_attribute('href')
        link_list.append(href)

# Extract initial data
extract_data()

while True:
    try:
        # Identify and click the 'Next' button
        current_page = driver.find_element(By.CSS_SELECTOR, "div[class='DC_117_pageTabs DC_117_pageActive']")
        driver.execute_script("arguments[0].scrollIntoView();", current_page)

        next_page_number = int(current_page.text) + 1

        next_page = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="root"]/div/div[4]/div[12]/div[{next_page_number}]'))
        )
        next_page.click()

        time.sleep(10)  
        extract_data()  # Extract data from the next page
    except Exception as e:
    
        break  # Exit the loop if the next button is not found or an error occurs

all_data = []

for route, link in zip(bus_routes_list, link_list):
    driver.get(link)
    time.sleep(5)
    
    try:
        view_buses_buttons = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']"))
        )

        # Click each "View Buses" button using JavaScript
        for view_buses_button in view_buses_buttons:
            driver.execute_script("arguments[0].scrollIntoView(true);", view_buses_button)
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2) 
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        bus_names = driver.find_elements(By.CSS_SELECTOR, 'div[class="travels lh-24 f-bold d-color"]')
        bus_types = driver.find_elements(By.CSS_SELECTOR, 'div[class="bus-type f-12 m-top-16 l-color evBus"]')
        starting_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="dp-time f-19 d-color f-bold"]')
        drop_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="bp-time f-19 d-color disp-Inline"]')
        ratings = driver.find_elements(By.CSS_SELECTOR, 'div[class="rating-sec lh-24"]')
        fares = driver.find_elements(By.CSS_SELECTOR, 'div[class="fare d-block"]')
        seats_available = driver.find_elements(By.CSS_SELECTOR, "div[class='seat-left m-top-30']")
        bus_durations = driver.find_elements(By.CSS_SELECTOR, 'div[class="dur l-color lh-24"]')

        list_lengths = [len(bus_names), len(bus_types), len(starting_times), len(drop_times), len(ratings), len(fares), len(seats_available), len(bus_durations)]
        min_length = min(list_lengths)

        if min_length == 0:
            continue

        for i in range(min_length):
            data= {
                "Busroutes": route,
                "Bus Routes Link": link,
                "Bus Name": bus_names[i].text if i < len(bus_names) else "NA",
                "Bus_Type": bus_types[i].text if i < len(bus_types) else "NA",
                "Starting_Time": starting_times[i].text if i < len(starting_times) else "NA",
                "Rating": ratings[i].text if i < len(ratings) and ratings[i].text else "NA",
                "Dropping_Time": drop_times[i].text if i < len(drop_times) else "NA",
                "Fare": fares[i].text if i < len(fares) else "NA",
                "Available Seats": seats_available[i].text if i < len(seats_available) and seats_available[i].text else "NA",
                "Duration": bus_durations[i].text if i < len(bus_durations) else "NA"
            }
            all_data.append(data)

    except Exception as e:
        pass



# Close the driver
driver.quit()


print("Successfully completed")

Successfully completed


In [12]:
data_6= pd.DataFrame(all_data)
data_6

,Busroutes,Bus Routes Link,Bus Name,Bus_Type,Starting_Time,Rating,Dropping_Time,Fare,Available Seats,Duration
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 532,Ordinary 3+2 Non AC Seater,17:59,2.0,04:15,INR 512,27 Seats available,10h 16m
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 538,Ordinary 3+2 Non AC Seater,18:31,4.0,05:30,INR 507,23 Seats available,10h 59m
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 93,Himsuta AC Seater Volvo/Scania 2+2,19:35,3.7,04:45,INR 892,14 Seats available,09h 10m
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 301,Ordinary 3+2 Non AC Seater,19:43,4.1,05:30,INR 507,27 Seats available,09h 47m
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 556,Ordinary 3+2 Non AC Seater,20:10,4.5,05:45,INR 507,7 Seats available,09h 35m
...,...,...,...,...,...,...,...,...,...,...
938,Kangra to Delhi,https://www.redbus.in/bus-tickets/kangra-to-delhi,Zingbus Plus,Volvo Multi-Axle A/C Semisleeper (2+2),22:05,4.5,06:55,634,25 Seats available,08h 50m
939,Kangra to Delhi,https://www.redbus.in/bus-tickets/kangra-to-delhi,New Himalaya Travels,Volvo A/C Semi Sleeper (2+2),20:10,4.5,06:20,INR 550,30 Seats available,10h 10m
940,Kangra to Delhi,https://www.redbus.in/bus-tickets/kangra-to-delhi,City Land Travels,Volvo A/C Semi Sleeper (2+2),20:40,4.2,06:00,540,34 Seats available,09h 20m
941,Kangra to Delhi,https://www.redbus.in/bus-tickets/kangra-to-delhi,New Himalaya Travels,Volvo A/C Semi Sleeper (2+2),20:45,4.5,06:15,INR 600,36 Seats available,09h 30m


In [35]:
#uttar pradesh
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

# Initialize the Chrome driver
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/")
time.sleep(3)
driver.maximize_window()
driver.get("https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile")
time.sleep(2)

# store data
bus_routes_list = []
link_list = []

def extract_data():
    # Extract data from the page
    bus_routes = driver.find_elements(By.CLASS_NAME, "route")
    for route in bus_routes:
        bus_routes_list.append(route.text)
    
    for link in bus_routes:
        href = link.get_attribute('href')
        link_list.append(href)

# Extract initial data
extract_data()

while True:
    try:
        # Identify and click the 'Next' button
        current_page = driver.find_element(By.CSS_SELECTOR, "div[class='DC_117_pageTabs DC_117_pageActive']")
        driver.execute_script("arguments[0].scrollIntoView();", current_page)

        next_page_number = int(current_page.text) + 1

        next_page = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="root"]/div/div[4]/div[12]/div[{next_page_number}]'))
        )
        next_page.click()

        time.sleep(10)  
        extract_data()  # Extract data from the next page
    except Exception as e:
        break  # Exit the loop if the next button is not found or an error occurs

all_data = []

for route, link in zip(bus_routes_list, link_list):
    driver.get(link)
    time.sleep(5)
    
    try:
        view_buses_buttons = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']"))
        )

        # Click each "View Buses" button using JavaScript
        for view_buses_button in view_buses_buttons:
            driver.execute_script("arguments[0].scrollIntoView(true);", view_buses_button)
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2) 
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        bus_names = driver.find_elements(By.CSS_SELECTOR, 'div[class="travels lh-24 f-bold d-color"]')
        bus_types = driver.find_elements(By.CSS_SELECTOR, 'div[class="bus-type f-12 m-top-16 l-color evBus"]')
        starting_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="dp-time f-19 d-color f-bold"]')
        drop_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="bp-time f-19 d-color disp-Inline"]')
        ratings = driver.find_elements(By.CSS_SELECTOR, 'div[class="rating-sec lh-24"]')
        fares = driver.find_elements(By.CSS_SELECTOR, 'div[class="fare d-block"]')
        seats_available = driver.find_elements(By.CSS_SELECTOR, "div[class='seat-left m-top-30']")
        bus_durations = driver.find_elements(By.CSS_SELECTOR, 'div[class="dur l-color lh-24"]')

        list_lengths = [len(bus_names), len(bus_types), len(starting_times), len(drop_times), len(ratings), len(fares), len(seats_available), len(bus_durations)]
        min_length = min(list_lengths)

        if min_length == 0:
            continue

        for i in range(min_length):
            data= {
                "Busroutes": route,
                "Bus Routes Link": link,
                "Bus Name": bus_names[i].text if i < len(bus_names) else "NA",
                "Bus_Type": bus_types[i].text if i < len(bus_types) else "NA",
                "Starting_Time": starting_times[i].text if i < len(starting_times) else "NA",
                "Rating": ratings[i].text if i < len(ratings) and ratings[i].text else "NA",
                "Dropping_Time": drop_times[i].text if i < len(drop_times) else "NA",
                "Fare": fares[i].text if i < len(fares) else "NA",
                "Available Seats": seats_available[i].text if i < len(seats_available) and seats_available[i].text else "NA",
                "Duration": bus_durations[i].text if i < len(bus_durations) else "NA"
            }
            all_data.append(data)

    except Exception as e:
        pass


driver.quit()

print("Successfully completed")

Successfully completed


In [13]:

data_7= pd.DataFrame(all_data)
data_7



,Busroutes,Bus Routes Link,Bus Name,Bus_Type,Starting_Time,Rating,Dropping_Time,Fare,Available Seats,Duration
0,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - RKD0130,Janrath AC Seater 2+3,17:31,2.0,22:31,INR 520,52 Seats available,05h 00m
1,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - BLY0111,Janrath AC Seater 2+2,19:01,3.2,23:45,INR 598,35 Seats available,04h 44m
2,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - HDD0063,Ordinary Non AC Seater 2+3,19:39,3.3,01:03,INR 415,52 Seats available,05h 24m
3,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - RKD0128,Janrath AC Seater 2+2,20:01,4.1,01:01,INR 598,36 Seats available,05h 00m
4,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - STD0354,Ordinary Non AC Seater 2+3,21:27,3.3,03:42,INR 448,37 Seats available,06h 15m
...,...,...,...,...,...,...,...,...,...,...
687,Lucknow to Gorakhpur (uttar pradesh),https://www.redbus.in/bus-tickets/lucknow-to-g...,RYS Travels,Bharat Benz A/C Seater / Sleeper (2+2),23:30,2.6,05:00,1169,35 Seats available,05h 30m
688,Lucknow to Gorakhpur (uttar pradesh),https://www.redbus.in/bus-tickets/lucknow-to-g...,Samay Shatabdi Travels Pvt Ltd,AC Sleeper (2+1),21:00,2.5,01:15,INR 699,11 Seats available,04h 15m
689,Lucknow to Gorakhpur (uttar pradesh),https://www.redbus.in/bus-tickets/lucknow-to-g...,Maan Travels,NON AC Seater / Sleeper 2+1,23:55,1.6,04:00,INR 1100,41 Seats available,04h 05m
690,Lucknow to Gorakhpur (uttar pradesh),https://www.redbus.in/bus-tickets/lucknow-to-g...,Bhumi Travels,A/C Seater / Sleeper (2+2),23:25,1.2,04:40,INR 1699,38 Seats available,05h 15m


In [37]:
#west bengal
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

# Initialize the Chrome driver
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/")
time.sleep(3)
driver.maximize_window()
driver.get("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile")
time.sleep(2)

# Lists to store data
bus_routes_list = []
link_list = []

def extract_data():
    # Extract data from the page
    bus_routes = driver.find_elements(By.CLASS_NAME, "route")
    for route in bus_routes:
        bus_routes_list.append(route.text)
    
    for link in bus_routes:
        href = link.get_attribute('href')
        link_list.append(href)

# Extract initial data
extract_data()

while True:
    try:
        # Identify and click the 'Next' button
        current_page = driver.find_element(By.CSS_SELECTOR, "div[class='DC_117_pageTabs DC_117_pageActive']")
        driver.execute_script("arguments[0].scrollIntoView();", current_page)

        next_page_number = int(current_page.text) + 1

        next_page = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="root"]/div/div[4]/div[12]/div[{next_page_number}]'))
        )
        next_page.click()

        time.sleep(10)  
        extract_data()  # Extract data from the next page
    except Exception as e:
        break  # Exit the loop if the next button is not found or an error occurs

all_data = []

for route, link in zip(bus_routes_list, link_list):
    driver.get(link)
    time.sleep(5)
    
    try:
        view_buses_buttons = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']"))
        )

        # Click each "View Buses" button using JavaScript
        for view_buses_button in view_buses_buttons:
            driver.execute_script("arguments[0].scrollIntoView(true);", view_buses_button)
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2) 
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        bus_names = driver.find_elements(By.CSS_SELECTOR, 'div[class="travels lh-24 f-bold d-color"]')
        bus_types = driver.find_elements(By.CSS_SELECTOR, 'div[class="bus-type f-12 m-top-16 l-color evBus"]')
        starting_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="dp-time f-19 d-color f-bold"]')
        drop_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="bp-time f-19 d-color disp-Inline"]')
        ratings = driver.find_elements(By.CSS_SELECTOR, 'div[class="rating-sec lh-24"]')
        fares = driver.find_elements(By.CSS_SELECTOR, 'div[class="fare d-block"]')
        seats_available = driver.find_elements(By.CSS_SELECTOR, "div[class='seat-left m-top-30']")
        bus_durations = driver.find_elements(By.CSS_SELECTOR, 'div[class="dur l-color lh-24"]')

        list_lengths = [len(bus_names), len(bus_types), len(starting_times), len(drop_times), len(ratings), len(fares), len(seats_available), len(bus_durations)]
        min_length = min(list_lengths)

        if min_length == 0:
            continue

        for i in range(min_length):
            data= {
                "Busroutes": route,
                "Bus Routes Link": link,
                "Bus Name": bus_names[i].text if i < len(bus_names) else "NA",
                "Bus_Type": bus_types[i].text if i < len(bus_types) else "NA",
                "Starting_Time": starting_times[i].text if i < len(starting_times) else "NA",
                "Rating": ratings[i].text if i < len(ratings) and ratings[i].text else "NA",
                "Dropping_Time": drop_times[i].text if i < len(drop_times) else "NA",
                "Fare": fares[i].text if i < len(fares) else "NA",
                "Available Seats": seats_available[i].text if i < len(seats_available) and seats_available[i].text else "NA",
                "Duration": bus_durations[i].text if i < len(bus_durations) else "NA"
            }
            all_data.append(data)

    except Exception as e:
        pass


# Close the driver
driver.quit()

print("successfully completed")



successfully completed


In [14]:

data_8= pd.DataFrame(all_data)
data_8


,Busroutes,Bus Routes Link,Bus Name,Bus_Type,Starting_Time,Rating,Dropping_Time,Fare,Available Seats,Duration
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) HABRA-DIGHA via Esplanade - 176|04:00,Non AC Seater (2+3),04:00,3.6,09:50,INR 165,39 Seats available,05h 50m
1,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) BARASAT-DIGHA via Bally - 38|04:45,Non AC Seater (2+3),04:45,3.9,09:35,INR 161,30 Seats available,04h 50m
2,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) BARASAT-DIGHA via Bally - 38|05:40,Non AC Seater (2+3),05:40,3.7,10:10,INR 161,21 Seats available,04h 30m
3,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) BADURIA - DIGHA via Bally - 155|09:00,Non AC Seater (2+3),09:00,3.6,12:25,INR 161,45 Seats available,03h 25m
4,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) HABRA-DIGHA via Bally - 26|10:00,Non AC Seater (2+3),10:00,4.3,14:45,INR 161,75 Seats available,04h 45m
...,...,...,...,...,...,...,...,...,...,...
199,Habra to Kolaghat,https://www.redbus.in/bus-tickets/habra-to-kol...,WBTC (CTC) Habra-Midnapur - 194|05:25,Non AC Seater (2+3),05:25,3.8,08:05,INR 87,52 Seats available,02h 40m
200,Habra to Contai (Kanthi),https://www.redbus.in/bus-tickets/habra-to-con...,WBTC (CTC) HABRA-DIGHA via Bally - 26|03:45,Non AC Seater (2+3),03:45,3.5,07:35,INR 149,52 Seats available,03h 50m
201,Habra to Contai (Kanthi),https://www.redbus.in/bus-tickets/habra-to-con...,WBTC (CTC) HABRA-DIGHA via Bally - 26|04:15,Non AC Seater (2+3),04:15,3.1,08:05,INR 149,47 Seats available,03h 50m
202,Habra to Contai (Kanthi),https://www.redbus.in/bus-tickets/habra-to-con...,WBTC (CTC) HABRA-DIGHA via Bally - 26|04:30,Non AC Seater (2+3),04:30,3.5,08:40,INR 149,52 Seats available,04h 10m


In [16]:
#chandigarh
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

# Initialize the Chrome driver
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/")
time.sleep(3)
driver.maximize_window()
driver.get("https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu")
time.sleep(2)

# Lists to store data
bus_routes_list = []
link_list = []

def extract_data():
    # Extract data from the page
    bus_routes = driver.find_elements(By.CLASS_NAME, "route")
    for route in bus_routes:
        bus_routes_list.append(route.text)
    
    for link in bus_routes:
        href = link.get_attribute('href')
        link_list.append(href)

# Extract initial data
extract_data()

while True:
    try:
        # Identify and click the 'Next' button
        current_page = driver.find_element(By.CSS_SELECTOR, "div[class='DC_117_pageTabs DC_117_pageActive']")
        driver.execute_script("arguments[0].scrollIntoView();", current_page)

        next_page_number = int(current_page.text) + 1

        next_page = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="root"]/div/div[4]/div[12]/div[{next_page_number}]'))
        )
        next_page.click()

        time.sleep(10)  
        extract_data()  # Extract data from the next page
    except Exception as e:
        break  # Exit the loop if the next button is not found or an error occurs

all_data = []

for route, link in zip(bus_routes_list, link_list):
    driver.get(link)
    time.sleep(5)
    
    try:
        view_buses_buttons = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']"))
        )

        # Click each "View Buses" button using JavaScript
        for view_buses_button in view_buses_buttons:
            driver.execute_script("arguments[0].scrollIntoView(true);", view_buses_button)
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2) 
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        bus_names = driver.find_elements(By.CSS_SELECTOR, 'div[class="travels lh-24 f-bold d-color"]')
        bus_types = driver.find_elements(By.CSS_SELECTOR, 'div[class="bus-type f-12 m-top-16 l-color evBus"]')
        starting_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="dp-time f-19 d-color f-bold"]')
        drop_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="bp-time f-19 d-color disp-Inline"]')
        ratings = driver.find_elements(By.CSS_SELECTOR, 'div[class="rating-sec lh-24"]')
        fares = driver.find_elements(By.CSS_SELECTOR, 'div[class="fare d-block"]')
        seats_available = driver.find_elements(By.CSS_SELECTOR, "div[class='seat-left m-top-30']")
        bus_durations = driver.find_elements(By.CSS_SELECTOR, 'div[class="dur l-color lh-24"]')

        list_lengths = [len(bus_names), len(bus_types), len(starting_times), len(drop_times), len(ratings), len(fares), len(seats_available), len(bus_durations)]
        min_length = min(list_lengths)

        if min_length == 0:
            continue

        for i in range(min_length):
            data= {
                "Busroutes": route,
                "Bus Routes Link": link,
                "Bus Name": bus_names[i].text if i < len(bus_names) else "NA",
                "Bus_Type": bus_types[i].text if i < len(bus_types) else "NA",
                "Starting_Time": starting_times[i].text if i < len(starting_times) else "NA",
                "Rating": ratings[i].text if i < len(ratings) and ratings[i].text else "NA",
                "Dropping_Time": drop_times[i].text if i < len(drop_times) else "NA",
                "Fare": fares[i].text if i < len(fares) else "NA",
                "Available Seats": seats_available[i].text if i < len(seats_available) and seats_available[i].text else "NA",
                "Duration": bus_durations[i].text if i < len(bus_durations) else "NA"
            }
            all_data.append(data)

    except Exception as e:
        pass


# Close the driver
driver.quit()

print("successfully completed")



successfully completed


In [17]:

data_9= pd.DataFrame(all_data)
data_9


,Busroutes,Bus Routes Link,Bus Name,Bus_Type,Starting_Time,Rating,Dropping_Time,Fare,Available Seats,Duration
0,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,Chandigarh Transport Undertaking (CTU) - 165750,HVAC Seater (2+3),12:20,3.2,15:05,INR 185.72,29 Seats available,02h 45m
1,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,Chandigarh Transport Undertaking (CTU) - 165752,HVAC Seater (2+3),14:20,3.7,17:05,INR 185.72,NA,02h 45m
2,Chandigarh to Shimla,https://www.redbus.in/bus-tickets/chandigarh-t...,,,,NA,,,47 Seats available,
3,Chandigarh to Yamuna Nagar,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165751,HVAC Seater (2+3),10:50,3.9,13:35,INR 185.72,38 Seats available,02h 45m
4,Chandigarh to Yamuna Nagar,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165753,HVAC Seater (2+3),15:40,4.0,18:25,INR 185.72,36 Seats available,02h 45m
...,...,...,...,...,...,...,...,...,...,...
73,Baijnath to Chandigarh,https://www.redbus.in/bus-tickets/baijnath-to-...,Chandigarh Transport Undertaking (CTU) - 165826,HVAC Seater (2+3),18:40,4.8,03:40,INR 592.39,NA,09h 00m
74,Baijnath to Chandigarh,https://www.redbus.in/bus-tickets/baijnath-to-...,Laxmi holidays,Volvo A/C Semi Sleeper (2+2),17:30,4.8,01:15,569,NA,07h 45m
75,Baijnath to Chandigarh,https://www.redbus.in/bus-tickets/baijnath-to-...,Northern Travels,Volvo A/C Semi Sleeper (2+2),18:00,4.1,01:00,INR 600,NA,07h 00m
76,Chandigarh to Jawala Ji,https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 1564,Ordinary 3+2 Non AC Seater,12:15,3.9,17:30,INR 300,37 Seats available,05h 15m


In [43]:
#panjab

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

# Initialize the Chrome driver
driver = webdriver.Chrome()
driver.get("https://www.redbus.in/")
time.sleep(3)
driver.maximize_window()
driver.get("https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile")
time.sleep(2)

# Lists to store data
bus_routes_list = []
link_list = []

def extract_data():
    # Extract data from the page
    bus_routes = driver.find_elements(By.CLASS_NAME, "route")
    for route in bus_routes:
        bus_routes_list.append(route.text)
    
    for link in bus_routes:
        href = link.get_attribute('href')
        link_list.append(href)

# Extract initial data
extract_data()

while True:
    try:
        # Identify and click the 'Next' button
        current_page = driver.find_element(By.CSS_SELECTOR, "div[class='DC_117_pageTabs DC_117_pageActive']")
        driver.execute_script("arguments[0].scrollIntoView();", current_page)

        next_page_number = int(current_page.text) + 1

        next_page = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="root"]/div/div[4]/div[12]/div[{next_page_number}]'))
        )
        next_page.click()

        time.sleep(10)  # Wait for the next page to load
        extract_data()  # Extract data from the next page
    except Exception as e:
        # print(f"Error exception: {e}")
        break  # Exit the loop if the next button is not found or an error occurs

all_data = []

for route, link in zip(bus_routes_list, link_list):
    driver.get(link)
    time.sleep(5)
    
    try:
        view_buses_buttons = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']"))
        )

        # Click each "View Buses" button using JavaScript
        for view_buses_button in view_buses_buttons:
            driver.execute_script("arguments[0].scrollIntoView(true);", view_buses_button)
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2) 
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        bus_names = driver.find_elements(By.CSS_SELECTOR, 'div[class="travels lh-24 f-bold d-color"]')
        bus_types = driver.find_elements(By.CSS_SELECTOR, 'div[class="bus-type f-12 m-top-16 l-color evBus"]')
        starting_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="dp-time f-19 d-color f-bold"]')
        drop_times = driver.find_elements(By.CSS_SELECTOR, 'div[class="bp-time f-19 d-color disp-Inline"]')
        ratings = driver.find_elements(By.CSS_SELECTOR, 'div[class="rating-sec lh-24"]')
        fares = driver.find_elements(By.CSS_SELECTOR, 'div[class="fare d-block"]')
        seats_available = driver.find_elements(By.CSS_SELECTOR, "div[class='seat-left m-top-30']")
        bus_durations = driver.find_elements(By.CSS_SELECTOR, 'div[class="dur l-color lh-24"]')

        list_lengths = [len(bus_names), len(bus_types), len(starting_times), len(drop_times), len(ratings), len(fares), len(seats_available), len(bus_durations)]
        min_length = min(list_lengths)

        if min_length == 0:
            # print(f"Skipping route {route} due to missing data")
            continue

        for i in range(min_length):
            data= {
                "Busroutes": route,
                "Bus Routes Link": link,
                "Bus Name": bus_names[i].text if i < len(bus_names) else "NA",
                "Bus_Type": bus_types[i].text if i < len(bus_types) else "NA",
                "Starting_Time": starting_times[i].text if i < len(starting_times) else "NA",
                "Rating": ratings[i].text if i < len(ratings) and ratings[i].text else "NA",
                "Dropping_Time": drop_times[i].text if i < len(drop_times) else "NA",
                "Fare": fares[i].text if i < len(fares) else "NA",
                "Available Seats": seats_available[i].text if i < len(seats_available) and seats_available[i].text else "NA",
                "Duration": bus_durations[i].text if i < len(bus_durations) else "NA"
            }
            all_data.append(data)

    except Exception as e:
        # print(f"An error occurred: {e}")
        pass


driver.quit()

print("successfully completed")

successfully completed


In [20]:

data_10 = pd.DataFrame(all_data)
data_10



,Busroutes,Bus Routes Link,Bus Name,Bus_Type,Starting_Time,Rating,Dropping_Time,Fare,Available Seats,Duration
0,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...,PEPSU (Punjab) - 67134,Volvo AC Seater (2+2),06:40,3.9,13:15,INR 845,37 Seats available,06h 35m
1,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...,PEPSU (Punjab) - 67161,Volvo AC Seater (2+2),10:15,3.3,17:15,INR 845,34 Seats available,07h 00m
2,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...,PEPSU (Punjab) - 67162,Volvo AC Seater (2+2),15:00,2.0,22:30,INR 845,34 Seats available,07h 30m
3,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...,PEPSU (Punjab) - 67096,Volvo AC Seater (2+2),16:10,4.0,23:10,INR 845,32 Seats available,07h 00m
4,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...,PEPSU (Punjab) - 67135,Volvo AC Seater (2+2),17:35,4.0,01:35,INR 845,28 Seats available,08h 00m
...,...,...,...,...,...,...,...,...,...,...
276,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...,PEPSU (Punjab) - 67128,Volvo AC Seater (2+2),20:50,3.9,04:50,INR 969,44 Seats available,08h 00m
277,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...,PEPSU (Punjab) - 67164,Volvo AC Seater (2+2),21:50,4.3,05:50,INR 969,37 Seats available,08h 00m
278,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),13:29,4.5,23:00,INR 3300,41 Seats available,09h 31m
279,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...,Indo Canadian Transport Pvt. Ltd.,A/C Sleeper (2+1),15:29,4.5,01:00,INR 3850,42 Seats available,09h 31m


In [38]:
total_df=pd.concat([data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10],ignore_index=True)
total_df

,Busroutes,Bus Routes Link,Bus Name,Bus_Type,Starting_Time,Rating,Dropping_Time,Fare,Available Seats,Duration
0,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 9356,Super Luxury (Non AC Seater 2+2 Push Back),11:30,4.0,18:05,INR 446,46 Seats available,06h 35m
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3858,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),11:30,3.6,16:30,INR 720,33 Seats available,05h 00m
2,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3601,Super Luxury (Non AC Seater 2+2 Push Back),12:00,2.5,20:20,INR 446,28 Seats available,08h 20m
3,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 35269,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),12:15,3.5,18:25,INR 683,43 Seats available,06h 10m
4,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3515,Super Luxury (Non AC Seater 2+2 Push Back),12:30,2.0,20:05,INR 446,25 Seats available,07h 35m
...,...,...,...,...,...,...,...,...,...,...
8892,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...,PEPSU (Punjab) - 67128,Volvo AC Seater (2+2),20:50,3.9,04:50,INR 969,44 Seats available,08h 00m
8893,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...,PEPSU (Punjab) - 67164,Volvo AC Seater (2+2),21:50,4.3,05:50,INR 969,37 Seats available,08h 00m
8894,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),13:29,4.5,23:00,INR 3300,41 Seats available,09h 31m
8895,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...,Indo Canadian Transport Pvt. Ltd.,A/C Sleeper (2+1),15:29,4.5,01:00,INR 3850,42 Seats available,09h 31m


In [39]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8897 entries, 0 to 8896
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Busroutes        8897 non-null   object
 1   Bus Routes Link  8897 non-null   object
 2   Bus Name         8897 non-null   object
 3   Bus_Type         8897 non-null   object
 4   Starting_Time    8897 non-null   object
 5   Rating           8897 non-null   object
 6   Dropping_Time    8897 non-null   object
 7   Fare             8897 non-null   object
 8   Available Seats  8897 non-null   object
 9   Duration         8897 non-null   object
dtypes: object(10)
memory usage: 695.2+ KB


Data cleaning

In [32]:
import numpy as np

In [ ]:
total_df['Available Seats'] = total_df['Available Seats'].str.extract('(\d)')  # Extract numeric part
total_df['Available Seats'] = total_df['Available Seats'].fillna(0)  # Replace NaN with 0
total_df['Available Seats'] = total_df['Available Seats'].astype(int)

In [50]:
df['Available Seats']

0       46
1       33
2       28
3       43
4       25
        ..
3894    36
3895    46
3896    48
3897    24
3898    21
Name: Available Seats, Length: 3899, dtype: int64

In [ ]:
total_df['Fare']=total_df['Fare'].str.replace("INR","")

total_df['Fare'] = total_df['Fare'].replace('', 0)

total_df['Fare']=total_df['Fare'].astype(float)

In [45]:
total_df['Fare']

0        446.0
1        720.0
2        446.0
3        683.0
4        446.0
         ...  
8892     969.0
8893     969.0
8894    3300.0
8895    3850.0
8896    3300.0
Name: Fare, Length: 8897, dtype: float64

In [ ]:
total_df['Duration'] = total_df['Duration'].str.replace('h', ':').str.replace('m', ':00').str.strip()


In [108]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8897 entries, 0 to 8896
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Busroutes        8897 non-null   object 
 1   Bus Routes Link  8897 non-null   object 
 2   Bus Name         8873 non-null   object 
 3   Bus_Type         8873 non-null   object 
 4   Starting_Time    8873 non-null   object 
 5   Rating           8897 non-null   float64
 6   Dropping_Time    8873 non-null   object 
 7   Fare             8897 non-null   float64
 8   Available Seats  8897 non-null   int64  
 9   Duration         8897 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 695.2+ KB


In [105]:
total_df.to_csv('Redbus.csv',index=False)

In [118]:
total_df

,Busroutes,Bus Routes Link,Bus Name,Bus_Type,Starting_Time,Rating,Dropping_Time,Fare,Available Seats,Duration
0,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 9356,Super Luxury (Non AC Seater 2+2 Push Back),11:30,4.0,06:05,446.0,4,06: 35:00
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3858,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),11:30,3.6,04:30,720.0,3,05: 00:00
2,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3601,Super Luxury (Non AC Seater 2+2 Push Back),12:00,2.5,08:20,446.0,2,08: 20:00
3,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 35269,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),12:15,3.5,06:25,683.0,4,06: 10:00
4,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3515,Super Luxury (Non AC Seater 2+2 Push Back),12:30,2.0,08:05,446.0,2,07: 35:00
...,...,...,...,...,...,...,...,...,...,...
8892,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...,PEPSU (Punjab) - 67128,Volvo AC Seater (2+2),08:50,3.9,04:50,969.0,4,08: 00:00
8893,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...,PEPSU (Punjab) - 67164,Volvo AC Seater (2+2),09:50,4.3,05:50,969.0,3,08: 00:00
8894,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),01:29,4.5,11:00,3300.0,4,09: 31:00
8895,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...,Indo Canadian Transport Pvt. Ltd.,A/C Sleeper (2+1),03:29,4.5,01:00,3850.0,4,09: 31:00
